# <B> Setup for SageMaker BYOM examples </B>
* Container: pytorch_p39

## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

## 1. Set roles

In [2]:
from sagemaker import get_execution_role

In [3]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

SageMaker Execution Role Name: AmazonSageMaker-ExecutionRole-20221206T163436


### 1.1 Attach IAM polich to sagemaker execution role (<b>with console</b>)
> step 1. IAM console 로 이동    
>
> step 2. 왼쪽 네비게이터에서  "Role" 선택  
> ![nn](images/Role.png)  
>
> step 3. SageMaker Execution Role 검색 후 role 선택 (상위 cell output 참조)  
> ![nn](images/search-by-rolename.png)  
>
> step 4. "attach policies" 메뉴 선택 
> ![nn](images/attach-policy-menu.png)  
>
> step 5. "IAMFullAccess" policy 검색 후 attach 
> ![nn](images/attach-policy.png) 

In [4]:
from utils.iam import iam_handler

In [5]:
iam = iam_handler()

In [6]:
strID = "DJ"
strPrefix = f'{strID}-SM-IMD'

### 1.2 Attach policy - sagemaker execution role
> SSM  
> S3 <BR>
> CodeCommit <BR>
> CodeBuild <BR>
> CodePipeline <BR>

In [10]:
listPolicyArn=[
    "arn:aws:iam::aws:policy/AmazonSSMFullAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    "arn:aws:iam::aws:policy/AWSCodeCommitFullAccess",
    "arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess",
    "arn:aws:iam::aws:policy/AWSCodePipeline_FullAccess",
]
for strPolicyArn in listPolicyArn: iam.attach_policy(strSageMakerRoleName, strPolicyArn)

### 1.3 Create role for lambda

In [11]:
listService = ["lambda.amazonaws.com"]
strRoleName = "-".join([strPrefix, "LabmdaRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole",
    "arn:aws:iam::aws:policy/AWSLambda_FullAccess",
    "arn:aws:iam::aws:policy/AmazonSageMakerFullAccess",
]
strDescription="Role for Lambda to call SageMaker functions'"

In [12]:
lambdaRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)

== CREATE ROLE ==
  Role Name: [DJ-SM-IMD-LabmdaRole] is already exist!!, so, this will be deleted and re-created.
{'ResponseMetadata': {'RequestId': '58aac7f5-dfd6-45b5-8b62-c8e6e585a4dc', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '58aac7f5-dfd6-45b5-8b62-c8e6e585a4dc', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Sun, 16 Apr 2023 06:40:11 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole
{'ResponseMetadata': {'RequestId': '99c516ad-607d-4181-87db-9c1d0bb71d90', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '99c516ad-607d-4181-87db-9c1d0bb71d90', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Sun, 16 Apr 2023 06:40:11 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AmazonSageMakerFullAccess
{'ResponseMetadata': {'RequestId': '5f0a5544-7678-4819-9e1c-41eb9bbe2208', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5f0a5544-7678-4819-9e1c-41eb9bbe2208', 'content-type': 

### 1.4 role-codebuild

In [13]:
listService = ["codebuild.amazonaws.com", "sagemaker.amazonaws.com"]
strRoleName = "-".join([strPrefix, "CodeBuildRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/AWSCodeCommitFullAccess",
    "arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    "arn:aws:iam::aws:policy/CloudWatchFullAccess",
    "arn:aws:iam::aws:policy/AmazonSageMakerFullAccess",
    "arn:aws:iam::aws:policy/AWSLambda_FullAccess",
    "arn:aws:iam::aws:policy/AmazonSSMFullAccess",
]
strDescription="Role for CodeBuild"

In [14]:
buildRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)
print (f"CODE-BUILD-ROLE-ARN: {buildRoleArn}")

== CREATE ROLE ==
  Role Name: [DJ-SM-IMD-CodeBuildRole] is already exist!!, so, this will be deleted and re-created.
{'ResponseMetadata': {'RequestId': '0986878d-4c95-48b1-8eba-6a04daa63f02', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0986878d-4c95-48b1-8eba-6a04daa63f02', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Sun, 16 Apr 2023 06:40:18 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AmazonSSMFullAccess
{'ResponseMetadata': {'RequestId': '12616141-4c4a-49c7-b68a-31fe01423d93', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '12616141-4c4a-49c7-b68a-31fe01423d93', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Sun, 16 Apr 2023 06:40:18 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess
{'ResponseMetadata': {'RequestId': '065fc8f0-e8a4-46dc-9cdc-dcb3ded58914', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '065fc8f0-e8a4-46dc-9cdc-dcb3ded58914', 'content-type': 'text/xml', 'content

### 1.5 role-codepipeline

In [15]:
listService = ["codepipeline.amazonaws.com", "codebuild.amazonaws.com"]
strRoleName = "-".join([strPrefix, "CodePipelineRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/AWSCodeCommitFullAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    "arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess",
    "arn:aws:iam::aws:policy/AWSCodeDeployFullAccess",
    "arn:aws:iam::aws:policy/AWSCodePipeline_FullAccess"
]
strDescription="Role for CodePipeline"

In [16]:
pipelineRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)

print (f"CODE-PIPELINE-ROLE-ARN: {pipelineRoleArn}")

== CREATE ROLE ==
  Role Name: [DJ-SM-IMD-CodePipelineRole] is already exist!!, so, this will be deleted and re-created.
{'ResponseMetadata': {'RequestId': '14c65c24-399b-47c4-812d-adf7a4c81a01', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '14c65c24-399b-47c4-812d-adf7a4c81a01', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Sun, 16 Apr 2023 06:40:24 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess
{'ResponseMetadata': {'RequestId': 'ac0bb909-ba42-48b5-97ae-817cbf0b868c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'ac0bb909-ba42-48b5-97ae-817cbf0b868c', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Sun, 16 Apr 2023 06:40:25 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AWSCodeCommitFullAccess
{'ResponseMetadata': {'RequestId': '7d777f30-1b3d-488e-ad94-c4874bfe7e3b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '7d777f30-1b3d-488e-ad94-c4874bfe7e3b', 'content-type': 'text/xml', '

## 2. Set default parameters

In [17]:
import boto3
import sagemaker
from utils.ssm import parameter_store

In [18]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)

In [19]:
pm.put_params(key="PREFIX", value=strPrefix, overwrite=True)
pm.put_params(key="-".join([strPrefix, "REGION"]), value=strRegionName, overwrite=True)
pm.put_params(key="-".join([strPrefix, "BUCKET"]), value=sagemaker.Session().default_bucket(), overwrite=True)
pm.put_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"]), value=get_execution_role(), overwrite=True)
pm.put_params(key="-".join([strPrefix, "ACCOUNT-ID"]), value=boto3.client("sts").get_caller_identity().get("Account"), overwrite=True)
pm.put_params(key="-".join([strPrefix, "LAMBDA-ROLE-ARN"]), value=lambdaRoleArn, overwrite=True)
pm.put_params(key="-".join([strPrefix, "CODE-BUILD-ROLE-ARN"]), value=buildRoleArn, overwrite=True)
pm.put_params(key="-".join([strPrefix, "CODE-PIPELINE-ROLE-ARN"]), value=pipelineRoleArn, overwrite=True)

'Store suceess'

In [20]:
print ("REGION: ", pm.get_params(key="-".join([strPrefix, "REGION"])))
print ("BUCKET: ", pm.get_params(key="-".join([strPrefix, "BUCKET"])))
print ("SAGEMAKER-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"])))
print ("ACCOUNT-ID: ", pm.get_params(key="-".join([strPrefix, "ACCOUNT-ID"])))
print ("LAMBDA-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "LAMBDA-ROLE-ARN"])))
print ("CODE-BUILD-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "CODE-BUILD-ROLE-ARN"])))
print ("CODE-PIPELINE-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "CODE-PIPELINE-ROLE-ARN"])))

REGION:  us-east-1
BUCKET:  sagemaker-us-east-1-419974056037
SAGEMAKER-ROLE-ARN:  arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436
ACCOUNT-ID:  419974056037
LAMBDA-ROLE-ARN:  arn:aws:iam::419974056037:role/DJ-SM-IMD-LabmdaRole
CODE-BUILD-ROLE-ARN:  arn:aws:iam::419974056037:role/DJ-SM-IMD-CodeBuildRole
CODE-PIPELINE-ROLE-ARN:  arn:aws:iam::419974056037:role/DJ-SM-IMD-CodePipelineRole
